# Import Required Library

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os 

# Get the Path for Images

In [ ]:
os.makedirs('./COVID19-DATASET/train/covid19')
os.makedirs('./COVID19-DATASET/train/normal')


In [ ]:
os.makedirs('./COVID19-DATASET/test/covid19')
os.makedirs('./COVID19-DATASET/test/normal')

In [ ]:
os.makedirs('./COVID19-DATASET/val/covid19')
os.makedirs('./COVID19-DATASET/val/normal')

In [ ]:
COVID_PATH = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'
NORMAL_PATH = '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'

**copy data from dataset in to train folder**

In [ ]:
from distutils.dir_util import copy_tree
fromDirectory= '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'
toDirectory='./COVID19-DATASET/train/covid19'
copy_tree(fromDirectory, toDirectory)

In [ ]:
fromDirectory= '../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'
toDirectory='./COVID19-DATASET/train/normal'
copy_tree(fromDirectory, toDirectory)

In [ ]:
fromDirectory= '../input/coronahack-chest-xraydataset/Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/train'
toDirectory='./COVID19-DATASET/train/covid19'
copy_tree(fromDirectory, toDirectory)

In [ ]:
fromDirectory= ('../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID'[:1000])
toDirectory='./COVID19-DATASET/test/covid19'
copy_tree(fromDirectory, toDirectory)

In [ ]:
fromDirectory= ('../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal'[:1000])
toDirectory='./COVID19-DATASET/test/normal'
copy_tree(fromDirectory, toDirectory)

**copy data from dataset in to validation folder**

In [ ]:
from distutils.dir_util import copy_tree
fromDirectory= ('../input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/NORMAL'[:200])
toDirectory='./COVID19-DATASET/val/normal'
copy_tree(fromDirectory, toDirectory)

In [ ]:
from distutils.dir_util import copy_tree
fromDirectory= ('../input/covid19-xray-images-using-cnn/images/test/corona'[:200])
toDirectory='./COVID19-DATASET/val/covid19'
copy_tree(fromDirectory, toDirectory)

In [ ]:
train_path = './COVID19-DATASET/train'
val_path = './COVID19-DATASET/val'
test_path = '../input/chest-xray-pneumonia/chest_xray/test/'
#test_path= test_data

# Define Constants

In [ ]:
# re-size all the images to a size VGG-16 expects.
IMAGE_SIZE = [224, 224]

# Set the batch size
BATCH_SIZE = 32  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001
NUM_CLASSES = 2 # We are aware of it.

In [ ]:
import os
CLASSES = os.listdir(train_path)
NUM_CLASSES = len(CLASSES)

In [ ]:
print("Class --> {} \n and the length is : {}".format(CLASSES, NUM_CLASSES))

# Load the Data / Images

## For Training dataset

In [ ]:
# Image Data Augmentation

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

In [ ]:
# Import the images from the train dataset.
# Make sure to provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(
    directory = train_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

## For Test Dataset

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Import the images from the test dataset.

test_set = test_datagen.flow_from_directory(
    directory = test_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

In [ ]:
# Import the VGG 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
### Sample... for adding Pooling (optional)
# global_average_layer = GlobalAveragePooling2D()

# prediction = Dense(NUM_CLASSES,activation='softmax')

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

prediction = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
history = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
model.save('my_model.h5')

# Evaluate the Model

In [ ]:
val_path = './COVID19-DATASET/val'

In [ ]:
# Generate Validation set.
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

In [ ]:
validation_steps = 200

loss0,accuracy0 = model.evaluate(validation_set, steps = validation_steps)

print("loss: {:.2f}".format(loss0))
print("accuracy: {:.2f}".format(accuracy0))

In [ ]:
# Generate Validation set.
validation_set2 = validation_datagen.flow_from_directory(
    directory = val_path,
    target_size = (224, 224),
    batch_size = 1,
    shuffle=False, 
    seed=42, 
    class_mode="binary"
)

# validation_set2.reset()

In [ ]:
# just capture the loss and accuray into val variable... unlike in pervious code to capture into loss0 and accuracy0. Just to showcase alternate way.

val = model.evaluate(validation_set, steps = validation_steps)

print("loss: {:.2f}".format(val[0]))
print("accuracy: {:.2f}".format(val[1]))

In [ ]:
# summarize history for loss

plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Validation (Test) loss')
plt.title('summarize history for loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('summarize history for accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predict

In [ ]:
# get sample image to test.
img_normal = image.load_img('../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-10.png', target_size = (224, 224))
img_pneumonia = image.load_img('../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1947_bacteria_4876.jpeg', target_size = (224, 224))

In [ ]:
def model_predict(img, actual):
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x_processed = preprocess_input(x)
    result = model.predict(x_processed)
    if(result[0][0]<.50):
        result="normal"
    else:
        result="corona positive"
        
    plt.figure()
    plt.imshow(img)
    plt.title('Actual : {} --> Predicted  : {}'.format(actual, result))
    
#     return result

In [ ]:
pred_normal = model_predict(img_normal, "normal")
pred_pneumonia = model_predict(img_pneumonia, "corona positive")

In [ ]:
img = image.load_img('../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/Normal-100.png', target_size = (224, 224))

pred = model_predict(img, "")